# RAG ToT  
Tree-of-Thought 기법을 적용

- RAG에 Tree-of-Thought(ToT) 기법을 적용하면,  
  단일 답변 생성이 아닌 **여러 추론 경로(후보 답변)를 생성·평가·선택**하는 구조로 확장할 수 있다.
- 각 Thought(노드)는 **조회된 문서 기반의 부분 추론 결과**를 의미하며,  
  이를 단계적으로 분기·탐색함으로써 답변의 정확성과 근거성을 높인다.
- 이를 통해 RAG는 단순 Q&A를 넘어  
  **복합 질문, 비교·분석형 질문에 대응 가능한 추론형 검색 시스템**으로 발전한다.

In [2]:
%pip install langchain langchain-openai -Uq

Note: you may need to restart the kernel to use updated packages.


In [3]:
from dotenv import load_dotenv
import os

load_dotenv()
os.environ['LANGSMITH_TRACING'] = 'true'
os.environ['LANGSMITH_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGSMITH_API_KEY'] = os.getenv('langsmith_key')
os.environ['LANGSMITH_PROJECT'] = 'skn23-langchain'
os.environ['OPENAI_API_KEY'] = os.getenv("openai_key")

In [5]:
from langchain_core.documents import Document  # LangChain Document 클래스 import

# 더미 벡터DB에서 경제 관련 Document 리스트를 반환하는 함수
def retrieve_vectordb(query=None):
    return [  # Document 객체 리스트 반환
        Document(  # 1번 문서 생성
            page_content="""  
정부는 2026년을 '한국경제 대도약 원년'으로 선포하고, 0%대까지 추락했던 잠재성장률을 다시 1.8% 이상으로 끌어올리기 위한 적극적 확장 재정 정책을 추진하고 있습니다.
기존의 감세 중심에서 '지출 중심'으로 전환하며, 내년도 성장률 목표 달성을 위한 대규모 예산 투입을 공식화했습니다.
규제 측면에서는 금산분리 완화 등 실용주의적 개혁을 추진하며, 한국은행은 2026년 1월 기준금리를 연 2.50%로 동결하며 신중한 기조를 유지 중입니다.
""",  # 문서 본문(정책 방향)
            metadata={  # 문서 메타데이터
                "source": "DOC1",  # 문서 ID/출처
                "title": "2026년 경제정책 방향",  # 문서 제목
                "category": "Policy"  # 문서 분류
            }
        ),
        Document(  # 2번 문서 생성
            page_content="""  
2026년 한국 경제는 수출 증가세가 둔화되는 가운데 내수가 완만한 회복세를 보이며 1.8% 수준의 성장을 기록할 것으로 전망됩니다.
이는 2025년의 저성장(0.7~0.8%)에서 벗어나 정상 궤도로 진입하는 과정입니다.
반도체, AI, 조선, 방산 분야는 긍정적이나 미·중 통상 갈등 및 지정학적 리스크로 인한 수출 불확실성은 여전히 높은 상황입니다.
""",  # 문서 본문(전망)
            metadata={  # 문서 메타데이터
                "source": "DOC2",  # 문서 ID/출처
                "title": "2026년 경제 전망",  # 문서 제목
                "category": "Forecast"  # 문서 분류
            }
        ),
        Document(  # 3번 문서 생성
            page_content="""  
2025년의 저성장 충격을 극복하기 위해 정부는 양극화 구조 타파와 지속 가능한 성장에 집중하고 있습니다.
대·중소기업 상생, 지역 균형 발전, 노동시장 이중구조 완화를 핵심 과제로 설정하였습니다.
또한 탄소중립, 에너지 전환 등 ESG 가치를 반영한 R&D 혁신과 AI 대전환을 통해 장기적인 국가 경쟁력 확보에 주력하고 있습니다.
""",  # 문서 본문(전략)
            metadata={  # 문서 메타데이터
                "source": "DOC3",  # 문서 ID/출처
                "title": "경기 침체 탈출 및 양극화 해소",  # 문서 제목
                "category": "Strategy"  # 문서 분류
            }
        )
    ]

# 데이터 확인
docs = retrieve_vectordb()  # 문서 리스트 로드(더미 벡터DB 조회)
docs

[Document(metadata={'source': 'DOC1', 'title': '2026년 경제정책 방향', 'category': 'Policy'}, page_content="  \n정부는 2026년을 '한국경제 대도약 원년'으로 선포하고, 0%대까지 추락했던 잠재성장률을 다시 1.8% 이상으로 끌어올리기 위한 적극적 확장 재정 정책을 추진하고 있습니다.\n기존의 감세 중심에서 '지출 중심'으로 전환하며, 내년도 성장률 목표 달성을 위한 대규모 예산 투입을 공식화했습니다.\n규제 측면에서는 금산분리 완화 등 실용주의적 개혁을 추진하며, 한국은행은 2026년 1월 기준금리를 연 2.50%로 동결하며 신중한 기조를 유지 중입니다.\n"),
 Document(metadata={'source': 'DOC2', 'title': '2026년 경제 전망', 'category': 'Forecast'}, page_content='  \n2026년 한국 경제는 수출 증가세가 둔화되는 가운데 내수가 완만한 회복세를 보이며 1.8% 수준의 성장을 기록할 것으로 전망됩니다.\n이는 2025년의 저성장(0.7~0.8%)에서 벗어나 정상 궤도로 진입하는 과정입니다.\n반도체, AI, 조선, 방산 분야는 긍정적이나 미·중 통상 갈등 및 지정학적 리스크로 인한 수출 불확실성은 여전히 높은 상황입니다.\n'),
 Document(metadata={'source': 'DOC3', 'title': '경기 침체 탈출 및 양극화 해소', 'category': 'Strategy'}, page_content='  \n2025년의 저성장 충격을 극복하기 위해 정부는 양극화 구조 타파와 지속 가능한 성장에 집중하고 있습니다.\n대·중소기업 상생, 지역 균형 발전, 노동시장 이중구조 완화를 핵심 과제로 설정하였습니다.\n또한 탄소중립, 에너지 전환 등 ESG 가치를 반영한 R&D 혁신과 AI 대전환을 통해 장기적인 국가 경쟁력 확보에 주력하고 있습니다.\n')]

In [ ]:
from langchain.chat_models import init_chat_model
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from pydantic import BaseModel, Field  # 구조화 출력 스키마 정의용
from typing import List, Literal       # 타입 힌트 (Literal은 라벨 제한)

# 전문가 1명의 분석 결과를 담는 스키마
class ExpertOpinion(BaseModel):
    role: str = Field(descrtion='전문가 역활 (예: 국제거시경제전문가)')
    analysis: str = Field(description='해당 전문가의 상세 분석내용')
    verdict: Literal['긍정', '부정', '중립'] = Field(description='분석 결과에 대한 종합 평가')

# 여러 전문가 의견 + 최종 종합 결론을 담는 스키마
class FinalOpinion(BaseModel):
    opinions: List[ExpertOpinion] = Field(description='개별 전문가들의 의견 리스트')
    final_analysis: str = Field(description='전문가의 의견을 종합한 최종 분석 내용')
    final_verdict: Literal['긍정', '부정', '중립'] = Field(description='전문가의 의견을 종합한 최종 평가')

llm = init_chat_model('openai:gpt-4.1-mini')
prompt = PromptTemplate.from_template('''  # 다중 관점 분석 및 결론 도출 프롬프트
당신은 현실세계의 복잡한 경제문제를 해결하기 위해 다양한 관점으로 바라보고, 분석하는 에이젼트입니다.
다음 세가지 관점에서 주어진 문서를 분석하고, 이를 종합하여 결론을 작성해주세요.

1. 분석가1 (국제거시경제): 글로벌 트렌드와 거시지표(성장률, 금리) 관점에서 분석 전문가 (긍정/중립/부정)
2. 분석가2 (지역경제/내수): 내수 소비, 건설, 지역경제활성화 관점에서 분석 전문가 (긍정/중립/부정)
3. 분석가3 (ESG/지속가능성): 장기적 안정성과 환경/사회적 영향 관점에서 분석 전문가 (긍정/중립/부정)

위 분석가들의 의견을 토대로 최종결론을 도출해주세요.  (긍정/중립/부정)

[문서]
{context}

[사용자 질문]
{query}
''')

chain = prompt | llm.with_structured_output(FinalOpinion)  # 응답을 FinalOpinion 스키마로 강제
query = '현재 한국 정부의 경제 정책을 어떻게 봐야 할까?'

context = '\n\n'.join([doc.page_content for doc in retrieve_vectordb(query)])  # 조회 문서 본문 1개의 문자열로 결합
final_opinion = chain.invoke({'query': query, 'context': context})
print(final_opinion)

C:\Users\Playdata\AppData\Local\Temp\ipykernel_2320\1689909673.py:8: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'descrtion'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  role: str = Field(descrtion='전문가 역활 (예: 국제거시경제전문가)')


opinions=[ExpertOpinion(role='분석가1 (국제거시경제)', analysis="한국 정부가 2026년을 '대도약 원년'으로 선포하고 잠재성장률을 1.8% 이상으로 끌어올리기 위한 확장 재정 정책을 추진하는 점은 글로벌 경제 둔화와 불확실성 속에서 성장 촉진을 위한 적극적인 신호이다. 수출 증가세 둔화와 지정학적 위험이 존재하지만, 금리를 동결하며 신중함을 유지하는 점도 긍정적이다. 따라서 거시경제 관점에서 정책은 전반적으로 긍정적이다.", verdict='긍정'), ExpertOpinion(role='분석가2 (지역경제/내수)', analysis='내수 회복세가 완만하고, 정부가 내수 및 지역경제 활성화를 위한 지출 중심 정책을 추진하는 점은 내수 경제의 재도약을 기대하게 한다. 양극화 해소와 지역 균형 발전 노력도 긍정적이나, 성장률이 아직 1.8%로 다소 낮은 점은 신중한 태도를 요구한다. 전체적으로 내수 및 지역경제 관점에서는 긍정과 중립의 중간 정도로 평가된다.', verdict='중립'), ExpertOpinion(role='분석가3 (ESG/지속가능성)', analysis='정부가 탄소중립, 에너지 전환, ESG 가치를 반영한 R&D 혁신과 AI 대전환에 주력하는 점은 장기적 안정성과 지속 가능성을 확보하려는 노력으로 높이 평가할 만하다. 대·중소기업 상생, 노동시장 이중구조 개선도 사회적 측면에서 긍정적이다. 따라서 ESG 및 지속가능성 관점에서는 분명히 긍정적이다.', verdict='긍정')] final_analysis='한국 정부의 경제 정책은 거시경제적 불확실성 속에서 성장률 회복을 목표로 하는 확장적이고 실용적인 접근을 취하면서도, 내수 회복과 지역경제 활성화에 신중하나 긍정적인 신호를 보이고 있다. 또한, ESG와 지속가능성 측면에서 혁신과 사회적 균형을 고려하는 점이 장기적 경쟁력 확보에 유리하다. 종합적으로 볼 때, 현 정책은 한국 경제의 정상 궤도 복귀를 위한 균형 잡힌 전략으로 평가할 수 있다.' final_

In [ ]:
# 개별 분석가의 의견
for opinion in final_opinion.opinions:
    print(f'[{opinion.role} 의견: {opinion.verdict}]')  # 전문가 역활/평가 라벨
    print(opinion.analysis)                             # 전문가 상세 분석
    print()

print('-' * 100)
print(f'[최종 의견: {final_opinion.final_verdict}]')   # 종합 평가 라벨 출력
print(f'{final_opinion.final_analysis}')              # 최종 종합 분석

[분석가1 (국제거시경제) 의견: 긍정]
한국 정부가 2026년을 '대도약 원년'으로 선포하고 잠재성장률을 1.8% 이상으로 끌어올리기 위한 확장 재정 정책을 추진하는 점은 글로벌 경제 둔화와 불확실성 속에서 성장 촉진을 위한 적극적인 신호이다. 수출 증가세 둔화와 지정학적 위험이 존재하지만, 금리를 동결하며 신중함을 유지하는 점도 긍정적이다. 따라서 거시경제 관점에서 정책은 전반적으로 긍정적이다.

[분석가2 (지역경제/내수) 의견: 중립]
내수 회복세가 완만하고, 정부가 내수 및 지역경제 활성화를 위한 지출 중심 정책을 추진하는 점은 내수 경제의 재도약을 기대하게 한다. 양극화 해소와 지역 균형 발전 노력도 긍정적이나, 성장률이 아직 1.8%로 다소 낮은 점은 신중한 태도를 요구한다. 전체적으로 내수 및 지역경제 관점에서는 긍정과 중립의 중간 정도로 평가된다.

[분석가3 (ESG/지속가능성) 의견: 긍정]
정부가 탄소중립, 에너지 전환, ESG 가치를 반영한 R&D 혁신과 AI 대전환에 주력하는 점은 장기적 안정성과 지속 가능성을 확보하려는 노력으로 높이 평가할 만하다. 대·중소기업 상생, 노동시장 이중구조 개선도 사회적 측면에서 긍정적이다. 따라서 ESG 및 지속가능성 관점에서는 분명히 긍정적이다.

----------------------------------------------------------------------------------------------------
[최종 의견: 긍정]
한국 정부의 경제 정책은 거시경제적 불확실성 속에서 성장률 회복을 목표로 하는 확장적이고 실용적인 접근을 취하면서도, 내수 회복과 지역경제 활성화에 신중하나 긍정적인 신호를 보이고 있다. 또한, ESG와 지속가능성 측면에서 혁신과 사회적 균형을 고려하는 점이 장기적 경쟁력 확보에 유리하다. 종합적으로 볼 때, 현 정책은 한국 경제의 정상 궤도 복귀를 위한 균형 잡힌 전략으로 평가할 수 있다.


## 다양한 ToT 예시


다음은 LLM이 Tree of Thoughts(ToT) 방식으로 여러 사고 분기를 탐색하도록 유도하는 프롬프트 예시이다.

각 프롬프트는 “분기 생성 → 평가 → 유망 분기 확장” 과정을 명시하도록 구성했다.

1. **수학 퍼즐**

   ```
   다음 수학 퍼즐을 해결할 때, 가능한 연산 조합(+, –, ×, ÷)을 모두 분기(branch)로 생성하고, 각 분기를 평가해 유망한 경로만 확장하여 최종 해답(24)을 찾아라.
   “2, 3, 5, 7을 이용해 24를 만들어라.”
   ```

2. **최단 경로 문제**

   ```
   아래 그래프에서 A에서 D로 가는 최단 경로를 찾을 때, 가능한 모든 다음 노드 선택지를 분기로 생성하고, 각 분기의 예상 비용을 평가한 뒤 유망한 경로를 우선 확장하라.
   노드: A–B:2, A–C:5, B–C:1, B–D:7, C–D:3
   ```

3. **일정 계획**

   ```
   1일 일정: ‘서울 관광’ 계획을 세울 때, 가능한 관광지 순서별 분기를 생성하고, 이동 시간과 선호도를 기준으로 각 분기를 평가하여 최적의 동선을 구성하라.
   관광지: 경복궁, 북촌한옥마을, 남산타워, 명동
   ```

4. **논리 퍼즐**

   ```
   세 사람(A, B, C)의 직업(의사, 변호사, 엔지니어)을 맞출 때, 각 가정(case)을 분기로 생성하고, 제시된 단서(“A는 의사가 아니며…” 등)에 따라 각 분기를 평가·필터링하여 정답을 찾아라.
   ```

5. **문장 완성 (창의적 글쓰기)**

   ```
   이야기의 다음 문장을 다각도로 이어갈 때, 가능한 전개 아이디어를 분기로 나열하고, 플롯의 흥미도와 일관성을 평가해 유망한 아이디어만 확장하여 이어쓰라.
   “주인공이 낯선 마을에 도착했을 때…”
   ```

6. **코드 디버깅**

   ```
   주어진 파이썬 코드의 버그 원인을 찾을 때, 가능한 원인 후보(변수 초기화 오류, 루프 조건 오류 등)를 분기로 생성하고, 로그 메시지·실행 결과를 평가해 유망한 원인만 탐색하여 수정 방안을 제시하라.
   ```

7. **상품 추천 시나리오**

   ```
   고객 프로필(20대 남성, 운동 애호가)을 기준으로 추천 아이템을 고를 때, 여러 추천 카테고리(런닝화, 요가매트, 스마트워치 등)를 분기로 생성하고, 예상 만족도 점수를 평가해 상위 2개 카테고리를 확장 추천하라.
   ```

8. **최적 배낭 문제(Knapsack)**

   ```
   무게 제한 15kg, 아이템 목록(각 무게·가치 정보)으로 배낭 구성을 할 때, 각 아이템 선택 여부를 분기로 생성하고, 현재 가치 합을 평가해 유망한 조합만 확장하여 최적 가치 조합을 찾아라.
   ```

9. **화학 합성 경로**

   ```
   목표 화합물 C의 합성 경로를 찾을 때, 가능한 반응 단계(반응 A, 반응 B, 반응 C 등)를 분기로 생성하고, 각 단계의 수율(yield)과 비용을 평가해 유망한 경로만 연쇄적으로 확장하라.
   ```

10. **디자인 의사결정**

   ```
   새로운 앱 UI 디자인을 할 때, 버튼 배치·색상·글꼴 조합을 분기로 생성하고, 사용성·브랜드 일관성 지표로 평가해 상위 3개 디자인 컨셉을 선택·확장하여 최종 시안을 제시하라.
   ```